In [8]:
import numpy as np
import pandas as pd
import cv2
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.get_logger().setLevel(logging.ERROR)

## Read The Data using tensorflow

In [9]:
train_data = image_dataset_from_directory("../input/face-mask-12k-images-dataset/Face Mask Dataset/Train",labels='inferred',
                                       label_mode='binary',interpolation='nearest',image_size=[150,150],batch_size=64,
                                       shuffle=True)
test_data = image_dataset_from_directory("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test",labels='inferred',
                                       label_mode='binary',interpolation='nearest',image_size=[150,150],batch_size=64,
                                       shuffle=True)
validation_data = image_dataset_from_directory("../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation",labels='inferred',
                                       label_mode='binary',interpolation='nearest',image_size=[150,150],batch_size=64,
                                       shuffle=True)

## Lets perform the data agumnetation to prevent the overfiting

In [10]:
aguObjTrain=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,rotation_range=45,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)
aguObjTest=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

In [43]:
trainGenerator = aguObjTrain.flow_from_directory(
        '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train',
        target_size=(256, 256),
        color_mode='rgb',
        batch_size=100,
        shuffle=True,
        class_mode='categorical')
testGenerator = aguObjTest.flow_from_directory(
        '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test',
        target_size=(256, 256),
        color_mode='rgb',
        batch_size=64,shuffle=True,
        class_mode='categorical')
validation_generator = aguObjTrain.flow_from_directory(
        '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation',
        target_size=(256, 256),
        color_mode='rgb',
        batch_size=64,shuffle=True,
        class_mode='categorical')



## Lets start transfer learning using mobilenet 

In [12]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [13]:
mobileNet = MobileNetV2(weights='imagenet',input_shape=(256,256,3),include_top=False)

In [14]:
#Freeze the layers weights 
for layer in mobileNet.layers:
    layer.trainable = False

In [15]:
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras import Sequential

In [21]:
model=Sequential()
model.add(mobileNet)
model.add(Flatten())
model.add(Dense(units=128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=2,activation='sigmoid'))

In [22]:
model.summary()

In [23]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [44]:
history=model.fit(trainGenerator,epochs=10,validation_data=validation_generator,verbose=1)

In [47]:
from keras.models import save_model
model.save('my_model')

In [50]:
plt.figure(figsize=(10,10))
plt.plot(history.history["accuracy"],label="accuracy")
plt.plot(history.history["val_accuracy"],label="val-accuracy")
plt.xlabel("epoches")
plt.ylabel("accuracy")
plt.legend()

In [49]:
model.evaluate(testGenerator)